# Ginger 2

In [ ]:
system:
You are an AI assistant designed to provide helpful, accurate, and concise answers to a wide range of questions. Your responses should be clear, friendly, and professional. If you encounter a question that you cannot answer, politely inform the user and suggest that they provide more specific details or ask a different question. Always prioritize providing correct information and be cautious with topics that require specialized knowledge or legal, medical, or financial advice. In such cases, remind users to consult with a qualified professional. Strive to understand the context of each query and adapt your responses to meet the user's needs
  


{% for item in chat_history %}
user:
{{item.inputs.question}}
assistant:
{{item.outputs.answer}}
{% endfor %}

user:
{{question}}
  

# flow.meta.yaml

In [ ]:
version: "1.0"
name: "Pesport AI Model Orchestration"
description: |
  A prompt flow that orchestrates text, image, and voice inputs for the Pesport app using GPT-4, DALL·E, and Whisper. Handles routing, ambiguity, and error management.
  
steps:
  # Step 1: Input Classification
  - step: 
      name: classify_input
      type: classifier
      description: |
        This step will classify whether the input is text, image, or voice.
      input: ${input_data}   # Raw input from user
      output: ${classification}
      classifier_rules:
        - rule:
            if: is_text(${input_data})
            then: "text"
        - rule:
            if: is_image(${input_data})
            then: "image"
        - rule:
            if: is_audio(${input_data})
            then: "audio"
        - rule:
            if: contains_text_and_image(${input_data})
            then: "text_only"  # Handles ambiguous cases like mixed input

  # Step 2: Route Input to GPT-4 (Text)
  - step: 
      name: process_text
      type: openai
      when: ${classification} == "text" || ${classification} == "text_only"
      description: "This step processes text input using GPT-4."
      model: gpt-4o-mini   # GPT-4 Model
      input:
        prompt: ${input_data}
        temperature: 0.8
        max_tokens: 800
        top_p: 0.95
        presence_penalty: 0
        frequency_penalty: 0
      output: ${gpt4_response}

  # Step 3: Route Input to DALL·E (Image)
  - step: 
      name: process_image
      type: openai
      when: ${classification} == "image"
      description: "This step processes image input using DALL·E."
      model: dalle
      input:
        prompt: ${input_data}
      output: ${dalle_response}

  # Step 4: Route Input to Whisper (Audio)
  - step: 
      name: process_audio
      type: openai
      when: ${classification} == "audio"
      description: "This step processes voice input using Whisper."
      model: whisper
      input:
        audio: ${input_data}
      output: ${whisper_response}

  # Step 5: Handling Multiple Models (Text + Image)
  - step: 
      name: process_mixed_input
      type: parallel
      when: ${classification} == "text_only"
      description: |
        This step will process the text portion of mixed input (text + image), ignoring the image for now.
      branches:
        - branch: 
            steps:
              - step: 
                  name: handle_text_part
                  type: openai
                  model: gpt-4o-mini
                  input:
                    prompt: extract_text(${input_data})
                    temperature: 0.8
                    max_tokens: 800
                    top_p: 0.95
                    presence_penalty: 0
                    frequency_penalty: 0
                  output: ${gpt4_response}

  # Step 6: Error Handling
  - step:
      name: error_handling
      type: error_handler
      description: |
        Logs and handles errors encountered during classification or model invocation.
      actions:
        - action: log_error(${classification_error})
        - action: respond_with_message("We encountered an issue with your input. Please try again.")
        
  # Step 7: Success Response
  - step:
      name: success_response
      type: response
      description: |
        Returns the final response from the appropriate model (GPT-4, DALL·E, or Whisper).
      when: ${classification} in ["text", "image", "audio", "text_only"]
      input:
        response_text: 
          - ${gpt4_response}
          - ${dalle_response}
          - ${whisper_response}
